#### DESCRIPTION: 

The file FBS.ipynb does all the pre-prcessing and cleanup needed to work with the FoodBalanceSheets_E_All_Data and the FoodBalanceSheetsHistoric_E_All_Data

Prior to the processing in Python, the following actions are taken in Excel for both files:
1) Add new columns "Non Perishable" and "Non Perishable Code" 
2) In the histroic file rename tbe below 
 - Groundnuts (Shelled Eq) renamed Groundnuts
 - Rice (Milled Equivalent) in historic renamed Rice and products
3) For food items in the below list, enter NP for the "Non Perishable" column and enter 1 in the "Non Perishable Code" column

Alcohol Non – Food
Alcoholic Beverages
Animal Fats
Barley and products
Beans
Beer
Beverages, Alcoholic
Beverages, Fermented
Cereals Excluding Beer
Cereals , Other
Cloves
Cocoa beans and Products
Coconut Oil
Coffee and Products
Cottonseed
Cottonseed oil
Fish, Body oil
Fish, liver oil
Groundnut Oil
Honey
Maize germ oil
Millets and Products
Miscellaneous
Nuts and Products
Oats
Oilcrops Oil, Other
Oilcrops, Other
OliveOil
Olives (including Preserved)
Palm Oil
Palm Kernel Oil
Pulses
Pulses, Other and products
Rape and Mustard Oil
Rape and Mustardseed
Rice and products
Ricebran Oil
Rye and products
Sesame seed
Sesameseed Oil
Sorghum and products
Soyabean Oil
Soyabeans
Spices
Spices, Other
Sugar & Sweeteners
Sugar (Raw Equivalent)
Sugar non-centrifugal
Sunflower seed
Sunflowerseed Oil
Sweeteners, Other
Tea (including mate)
Treenuts
Vegetable Oils
Wheat and products
Wine

4) For food items in the below list, enter P for the "Non Perishable" column and enter -1 in the "Non Perishable Code" column

Apples and Products
Aquatic Animals, Others
Aquatic Plants
Aquatic products, Others
Bananas
Bovine Meat
Butter, Ghee
Cassava and products
Cephalopods
Citrus, Other
Coconuts-incl Copra
Cream
Crustaceans
Dates
Demersal Fish
Eggs
Fats, Animals, Raw
Fish, seafood
Freshwater Fish
Fruits – Excluding wine
Fruits, Other
Grapefruit and products
Grapefruit and Products (excl Wine)
Groundnuts
Infant food
Lemons, Limes and Products
Maize and Products
Marine Fish, Other
Meat
Meat, Aquatic Mammals
Meat, Other
Milk – excluding Butter
Molluscs, Other
Mutton and Goat Meat
Oilcrops
Offals
Offals, Edible
Onions
Oranges, Mandarins
Palm Kernels
Peas
Pelagic Fish
Pepper
Pigmeat
Pimento
Pineapples and products
Plantains
Potatoes and products
Poultry Meat
Roots, Others
Starchy Roots
Stimulants
Sugar beet
Sugar cane
Sugar crops
Sweet Potatoes
Tomatoes and products
Vegetables
Vegetables, Other
Yams

5) Filter "Area" column only to be those countries listed in https://www.countries-ofthe-world.com/all-countries.html. Rename Swaziland to Eswatani in the historic file as the country was renamed in the recent years. If country doesn't exist today, remove it from the historic file as it will not be to visualize this country in the current world map

6) Filter "Element" column to only include entries for "Production" and "Losses". We are not interested in "Feed Use", "Seed Use" etc

7) Save the corresponding files as country_historic.csv and country_latest.csv. Now these can be merged by the code in the jupyter notebooks

#### INSTALLATION: 
Install pandas package if not already present using th below command

conda install -c anaconda pandas

#### EXECUTION:
Please ensure the country_latest.csv and country_historic.csv are saved in the same location as the notebook FBS_cleanup.ipynb. Then simply run the notebook.


In [1]:
import pandas as pd

In [2]:
#read the files
df1 = pd.read_csv("country_historic.csv")
df2 = pd.read_csv("country_latest.csv")

In [3]:
#sanity checks
#same items present in both 
# Groundnuts (Shelled Eq) in historic renamed Groundnuts
# Rice (Milled Equivalent) in historic renamed Rice and products
items_df1 = set(df1.Item.value_counts().index)
items_df2 = set(df2.Item.value_counts().index)
print(items_df2 - items_df1)
print(items_df1 - items_df2)

#all items have same perishable and non perishable codes

countries_df1 = set(df1.Area.value_counts().index)
countries_df2 = set(df2.Area.value_counts().index)
print(countries_df2 - countries_df1)
print(countries_df1 - countries_df2)

#Comoros, North Macedonia, Papua New Guinea, Seychelles, Brunei Darussalam - ok
#Swaziland renamed Eswatini
#'United Kingdom' renamed United Kingdom of Great Britain and Northern Ireland
#'Czechoslovakia' - not inluded in historic as split up as Czech and Slovakia in 1993
#USSR not included in historic as split up in 1992

set()
set()
{'Seychelles', 'Comoros', 'Papua New Guinea', 'North Macedonia'}
{'Brunei Darussalam', 'Czechoslovakia'}


In [4]:
df1.shape, df2.shape

((18004, 58), (22056, 10))

In [5]:
#prepare the main df with the primary keys, need to use both as some new wer countries are not present in historic file
primary_key = ['Area','Item','Non Perishable Code','Non Perishable','Element']

df = pd.concat([df1[primary_key], df2[primary_key]], axis = 0)

df.shape

(40060, 5)

In [6]:
df.head()


,Area,Item,Non Perishable Code,Non Perishable,Element
0,Afghanistan,Cereals - Excluding Beer,1,NP,Production
1,Afghanistan,Cereals - Excluding Beer,1,NP,Losses
2,Afghanistan,Wheat and products,1,NP,Production
3,Afghanistan,Wheat and products,1,NP,Losses
4,Afghanistan,Rice and products,1,NP,Production


In [7]:
df.drop_duplicates(inplace=True)
df.shape

(21767, 5)

In [8]:
df = pd.merge(left = df, right = df1, how = "left", on = primary_key)
df = pd.merge(left = df, right = df2, how = "left", on = primary_key)

In [9]:
#final merged dataset
df.head()

,Area,Item,Non Perishable Code,Non Perishable,Element,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,Cereals - Excluding Beer,1,NP,Production,3589.0,3590.0,3272.0,3605.0,3658.0,...,6299.0,5733.0,4457.0,6213.0,6350.0,6748.0,5816.0,5537.0,4900.0,4141.0
1,Afghanistan,Cereals - Excluding Beer,1,NP,Losses,309.0,307.0,287.0,316.0,320.0,...,881.0,784.0,604.0,865.0,883.0,963.0,838.0,801.0,707.0,591.0
2,Afghanistan,Wheat and products,1,NP,Production,2279.0,2279.0,1947.0,2230.0,2282.0,...,5064.0,4532.0,3388.0,5050.0,5169.0,5370.0,4673.0,4555.0,4281.0,3613.0
3,Afghanistan,Wheat and products,1,NP,Losses,185.0,183.0,162.0,187.0,191.0,...,768.0,678.0,508.0,758.0,775.0,803.0,701.0,684.0,642.0,542.0
4,Afghanistan,Rice and products,1,NP,Production,213.0,213.0,213.0,253.0,253.0,...,430.0,448.0,448.0,334.0,342.0,537.0,410.0,357.0,338.0,352.0


In [10]:
df.drop_duplicates(inplace=True)
df.shape
#df[(df.Area == 'Afghanistan') & (df.Item == 'Eggs')]

(21767, 63)

In [11]:
df.fillna(0, inplace=True)

In [12]:
cols_to_sum = [str(i) for i in range(1961, 2019)]
    
df["Total All Years"] = df[cols_to_sum].sum(axis=1)

df.head()


,Area,Item,Non Perishable Code,Non Perishable,Element,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total All Years
0,Afghanistan,Cereals - Excluding Beer,1,NP,Production,3589.0,3590.0,3272.0,3605.0,3658.0,...,5733.0,4457.0,6213.0,6350.0,6748.0,5816.0,5537.0,4900.0,4141.0,225996.0
1,Afghanistan,Cereals - Excluding Beer,1,NP,Losses,309.0,307.0,287.0,316.0,320.0,...,784.0,604.0,865.0,883.0,963.0,838.0,801.0,707.0,591.0,24697.0
2,Afghanistan,Wheat and products,1,NP,Production,2279.0,2279.0,1947.0,2230.0,2282.0,...,4532.0,3388.0,5050.0,5169.0,5370.0,4673.0,4555.0,4281.0,3613.0,161378.0
3,Afghanistan,Wheat and products,1,NP,Losses,185.0,183.0,162.0,187.0,191.0,...,678.0,508.0,758.0,775.0,803.0,701.0,684.0,642.0,542.0,18634.0
4,Afghanistan,Rice and products,1,NP,Production,213.0,213.0,213.0,253.0,253.0,...,448.0,448.0,334.0,342.0,537.0,410.0,357.0,338.0,352.0,16200.0


In [13]:
df.shape

(21767, 64)

In [14]:
df = df[df["Total All Years"] > 0]
df.shape

(17483, 64)

In [15]:
all_countries = df.Area.value_counts().index
frames = []
for country in all_countries:
    temp = df[df.Area == country]
    all_items = temp.Item.value_counts().index
    for item in all_items:
        new_temp = temp[temp.Item == item]
        if new_temp.shape[0] < 2:
            continue
        frames.append(new_temp)

df_cleaned = pd.concat(frames, axis = 0)
df_cleaned.shape

(11240, 64)

In [16]:
#create final merged csv for machine learning and analysis activities
df_cleaned.reset_index(drop = True, inplace=True)
df_cleaned.to_csv("country_merged.csv")


In [17]:
df_cleaned.head()

,Area,Item,Non Perishable Code,Non Perishable,Element,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total All Years
0,"China, mainland",Bananas,-1,P,Production,48.0,35.0,43.0,99.0,145.0,...,9561.0,10400.0,11558.0,12075.0,11792.0,10627.0,10940.0,11170.0,11222.0,202606.0
1,"China, mainland",Bananas,-1,P,Losses,5.0,4.0,4.0,10.0,15.0,...,2045.0,2244.0,2437.0,2518.0,2584.0,2340.0,2365.0,2442.0,2553.0,34829.0
2,"China, mainland",Vegetables,-1,P,Production,58165.0,54298.0,48045.0,43051.0,45651.0,...,542797.0,561453.0,567914.0,580702.0,576841.0,604867.0,614694.0,618719.0,624524.0,13516993.0
3,"China, mainland",Vegetables,-1,P,Losses,4273.0,4074.0,3581.0,3197.0,3404.0,...,44267.0,47906.0,42497.0,43406.0,47860.0,49286.0,50169.0,50682.0,47779.0,1073515.0
4,"China, mainland",Alcoholic Beverages,1,NP,Production,689.0,910.0,832.0,863.0,1004.0,...,58985.0,62455.0,61946.0,64828.0,66366.0,62484.0,60016.0,60077.0,53565.0,1333750.0
